In [42]:
import pandas as pd 


In [43]:
csv_mask_path = '/home/sposso22/Desktop/DATA_REVIEW/final_MASK_csv_files'
csv_mam_path = '/home/sposso22/Desktop/DATA_REVIEW/final_Full_mam_csv_files'
files = {'Mam_Set':csv_mam_path,'Mask_Set':csv_mask_path}
type_dict = ['Calc-Training','Mass-Training']


final_dfs =dict()
for path in files.keys():
    dfs=[]
    for i in type_dict:


        data = pd.read_csv(files[path]+'/'+i.lower()+'.csv')
        data = data.rename(columns={"pathology": "label"})


        if i.startswith("Calc"):

            labels = {"MALIGNANT":1 , "BENIGN":2}
            data["label"]= data.label.map(labels)

        else:

            labels = {"MALIGNANT":3 , "BENIGN":4}  
            data["label"]= data.label.map(labels)

 

        dfs.append(data)

    final_df = pd.concat(dfs)
    
    final_dfs[path] = final_df

In [45]:
mam_df = final_dfs['Mam_Set'].reset_index(drop= True)
final_mam_df = mam_df.drop(columns=["img"])
final_mam_df.to_csv(csv_mam_path+'/'+'official_mam_train_set.csv',index = False)
#Rename Columns 
mam_df = mam_df.rename(columns={"img": "merge","label":"label_mam"})
#Reset index 
mask_df =final_dfs['Mask_Set'].reset_index(drop= True)
#Generate  a new column in the mask dataframe with the original names of mammograms. 
#This columns  is equal to the  "img" column of the mammogram dataframe
mask_df['merge'] = mask_df.apply(lambda row: row['img'][:-2],
                                         axis=1) 
#Use the column created  in the previous line to match the mammograms paths with the mask paths
new_mask_df = pd.merge(mask_df, mam_df, how='inner',
                                     on='merge')

#Remove useless columns 
new_mask_df = new_mask_df.drop(columns=["img","merge","label_mam"])
#Change the order of columns 
new_mask_df = new_mask_df.iloc[:,[2,0,1]]
new_mask_df.to_csv("official_mask_train_set.csv",index=False)